In [ ]:
# Credit: https://www.kaggle.com/foolofatook/news-classification-using-bert
# CV credit: https://www.kaggle.com/code/ravi02516/bert-training-5-fold-cross-validation

In [ ]:
!pip uninstall -y transformers
!pip uninstall -y keras
!pip uninstall -y tensorflow
!pip install --upgrade keras==2.10.0
!pip install --upgrade tensorflow==2.10.0
!pip install --upgrade transformers==4.22.2
!pip install unidecode
# !pip install --upgrade keras==2.6.0
# !pip install --upgrade tensorflow==2.6.0
# !pip install --upgrade transformers==4.16.2
#!pip install pyyaml h5py

Found existing installation: keras 2.12.0
Uninstalling keras-2.12.0:
  Successfully uninstalled keras-2.12.0
Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 58.4 MB/s eta 0:00:00
  Attemptin

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
import transformers #huggingface transformers library
from transformers import TFAutoModel, AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.metrics import confusion_matrix

import re
from datetime import datetime as dt

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from unidecode import unidecode

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    #strategy = tf.distribute.experimental.TPUStrategy(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.29.107.242:8470
REPLICAS:  8


In [ ]:
# from google.colab import files
# import io
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)

# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/reGroundTruthBERT.csv', dtype='str')
#df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/CGT_ReGTR_BERT_Input.csv', dtype='str')
df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/data/GrounTruthBERT.csv', dtype='str', usecols=['TID', 'OrigTweet', 'InReplyTo', 'Target', 'Label'])
# df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/data/GrounTruthBERT.csv', dtype='str', usecols=['TID', 'CleanTweetNoHttp', 'InReplyTo', 'Target', 'Label'])
df.head()

,TID,OrigTweet,InReplyTo,Target,Label
0,826262311560216578,#coup has started. First of many steps. #rebel...,NaN,10,Politics
1,1193437298303438858,@MarkSZaidEsq @jody_prichard Funny you want to...,826262311560216578,10,Politics
2,1194280882540036098,"@MarkSZaidEsq at THAT time, the only ""stepping...",826262311560216578,10,Politics
3,1194635634960478208,@MarkSZaidEsq It's not a coup. It's a Constit...,826262311560216578,10,Politics
4,1194636052096413696,"@MarkSZaidEsq BTW, I've read your client HOSTE...",826262311560216578,10,Politics


In [ ]:
# Remove unlabelled data
df = df[df.Target.notna()]

In [ ]:
len(df)

124977

In [ ]:
# CGT: Remove Label:Health, Target:4
# df = df[df.Label != 'Health']

In [ ]:
# convert some columns to int
#df['CountReplyTweets'] = df['CountReplyTweets'].astype(float)
#df['CountReplyWords'] = df['CountReplyWords'].astype(float)
#df['CountReplyChars'] = df['CountReplyChars'].astype(float)
df['Target'] = df['Target'].astype(int)

In [ ]:
def remove1stMent(text):
  if text[0] == '@':
      words = text.split()
      words = words[1:] # remove the @mention word
      text = " ".join(words).lstrip()
  return(text)

def removeSpecialChar(text):
  #old text = re.sub(r'\W', ' ', text) #replace ALL non-word characters, including emojis with space
  # remove all non ASCII characters 

  # credit: https://stackoverflow.com/questions/2758921/regular-expression-that-finds-and-replaces-non-ascii-characters-with-python
  text = re.sub(r"[\u0080-\uFFFF]", " ", text) #see ASCII list: https://www.asciitable.com/
  text = " ".join(text.split()) # replace multiple spaces with a single space
  return(text)

def removeDigits(text):
  text = re.sub(r'\d', ' ', text) #replace digits with space
  return(text)

def cleanColloquials(text):
  #Replace common abbreviations and slangs
  text = text.replace(" i m "," i am ")
  text = text.replace(" i ve "," i have ")
  text = text.replace(" i ll "," i will ")
  text = text.replace(" i d "," i had ")
  text = text.replace(" that s "," that is ")
  text = text.replace(" isn t "," is not ")
  text = text.replace(" it s "," it is ")
  text = text.replace(" she s "," she is ")
  text = text.replace(" he s "," he is ")
  text = text.replace(" u "," you ")
  text = text.replace(" ur "," your ")
  text = text.replace(" b4 "," before ")
  text = text.replace(" wasnt "," was not ")
  text = text.replace(" wasn t "," was not ")
  text = text.replace(" cant "," can not ")
  text = text.replace(" can t "," can not ")
  text = text.replace(" couldnt "," could not ")
  text = text.replace(" couldn t "," could not ")
  text = text.replace(" wouldnt "," would not ")
  text = text.replace(" wouldn t "," would not ")
  text = text.replace(" dont "," do not ")
  text = text.replace(" don t "," do not ")
  text = text.replace(" didnt "," did not ")
  text = text.replace(" didn t "," did not ")
  text = text.replace(" let s "," let us ")
  text = text.replace(" i'm "," i am ")
  text = text.replace(" i've "," i have ")
  text = text.replace(" i'll "," i will ")
  text = text.replace(" i'd "," i had ")
  text = text.replace(" that's "," that is ")
  text = text.replace(" isn't "," is not ")
  text = text.replace(" it's "," it is ")
  text = text.replace(" she's "," she is ")
  text = text.replace(" he's "," he is ")
  text = text.replace(" u "," you ")
  text = text.replace(" ur "," your ")
  text = text.replace(" b4 "," before ")
  text = text.replace(" wasn't "," was not ")
  text = text.replace(" can't "," can not ")
  text = text.replace(" couldn't "," could not ")
  text = text.replace(" wouldn't "," would not ")
  text = text.replace(" don't "," do not ")
  text = text.replace(" didn't "," did not ")
  text = text.replace(" let's "," let us ")
  text = text.replace(" luv "," love ")
  text = text.replace(" true "," truth ")
  text = text.replace(" ppl "," people ")
  text = text.replace(" fb "," facebook ")
  text = text.replace(" b day "," birthday ")
  text = text.replace(" bday "," birthday ")
  if (len(text.strip())  == 0):
      text = ' ' #replace None with a single space
  return(text)

def removeHashtags(text):
  words = text.split()
  words = filter(lambda x:x[0]!='#', words)
  text = " ".join(words)
  if (len(text.strip())  == 0):
      text = ' ' #replace None with a single space
  return(text)

def removeMentions(text):
  words = text.split()
  words = filter(lambda x:x[0]!='@', words)
  text = " ".join(words)
  if (len(text.strip())  == 0):
      text = ' ' #replace None with a single space
  return(text)

def removeHttpWeb(text):
  words = text.split()
  words = filter(lambda x:x[0]!='&', words)
  words = filter(lambda x:x[0:4]!='http', words)
  text = " ".join(words)
  if (len(text.strip())  == 0):
      text = ' ' #replace None with a single space
  return(text)

def removeUnicode(text):
  #convert unicode chars to ascii
  text = unidecode(text)
  return(text)



In [ ]:
# CLEANUP FACTORY
# remove1stMent
# removeSpecialChar
# removeDigits
# cleanColloquials
# removeHashtags
# removeMentions
# removeHttpWeb
# removeUnicode

def cleanup(text):
  #Scenario1: 
  #text = removeUnicode(text)
  text = removeSpecialChar(text)
  text = remove1stMent(text)
  text = removeHttpWeb(text)
  text = cleanColloquials(text)

  #Scenario2: remove1stMent, removeHttpWeb, removeSpecialChar, cleanColloquials
  #Scenario3: remove1stMent, removeHttpWeb, removeHashtags, removeSpecialChar, cleanColloquials
  return(text)

In [ ]:
df['CleanTweet'] = df['OrigTweet'].apply(cleanup)
#remove1stMentNoSpecChar('@MarkSZaidEsq @tko From the evidence and more to co')
# df['CleanTweet'] = df['CleanTweetNoHttp'].apply(remove1stMentNoSpecChar)

In [ ]:
print(f"The dataset contains { df.Target.nunique() } unique categories")

The dataset contains 5 unique categories


In [ ]:
# convert the tweets into lower case if uncased model.
df['CleanTweet'] = df['CleanTweet'].apply(lambda x: str(x).lower())
# trim to 280 characters max
df['CleanTweet'] = df['CleanTweet'].str.slice(0,279)
# calculating the length of tweet
df['CleanTweet_len'] = df['CleanTweet'].apply(lambda x: len(str(x).split()))
# Remove tweets with less than 10 words
df = df.query('CleanTweet_len > 9')

In [ ]:
len(df)

101454

In [ ]:
df['Label'].unique()

array(['Politics', 'Entertainment', 'Business', 'Sports', 'Technology'],
      dtype=object)

In [ ]:
df['Target'].unique()

array([10,  6,  3, 11, 12])

In [ ]:
# reGTr: Change 6,International Politics and 8,National Politics to 6,Politics
# df.loc[(df.Label == 'International Politics'), 'Label'] = 'Politics'
# df.loc[(df.Label == 'National Politics'), 'Label'] = 'Politics'

In [ ]:
#For the initial Ground Truth:
df.loc[(df.Target == 10), 'Target'] = 0
df.loc[(df.Target == 6), 'Target'] = 1
df.loc[(df.Target == 3), 'Target'] = 2
df.loc[(df.Target == 11), 'Target'] = 3
df.loc[(df.Target == 12), 'Target'] = 4

#For reGroundTruth - change only 8,9 and 11
#df.loc[(df.Target == 8), 'Target'] = 0
#df.loc[(df.Target == 9), 'Target'] = 3
#df.loc[(df.Target == 11), 'Target'] = 4

# For CGT + reGTr
# df.loc[(df.Target == 1), 'Target'] = 0
# df.loc[(df.Target == 2), 'Target'] = 1
# df.loc[(df.Target == 3), 'Target'] = 2
# df.loc[(df.Target == 5), 'Target'] = 3
# df.loc[(df.Target == 6), 'Target'] = 4 #change old 6,International Politics to 4
# df.loc[(df.Target == 7), 'Target'] = 5
# df.loc[(df.Target == 8), 'Target'] = 4  #change old 8,National Politics to 4
# df.loc[(df.Target == 9), 'Target'] = 6
# df.loc[(df.Target == 10), 'Target'] = 7
# df.loc[(df.Target == 11), 'Target'] = 8
#df.loc[(df.Target == 11), 'Target'] = 10

In [ ]:
# set index to TID (used later in the Training/Test Series)
df.index = df['TID']
df.index.rename('gt_idx', inplace=True)

In [ ]:
df.head()

,TID,OrigTweet,InReplyTo,Target,Label,CleanTweet,CleanTweet_len
gt_idx,,,,,,,
826262311560216578,826262311560216578,#coup has started. First of many steps. #rebel...,NaN,0,Politics,#coup has started. first of many steps. #rebel...,13
1193437298303438858,1193437298303438858,@MarkSZaidEsq @jody_prichard Funny you want to...,826262311560216578,0,Politics,@jody_prichard funny you want to discriminate ...,47
1194280882540036098,1194280882540036098,"@MarkSZaidEsq at THAT time, the only ""stepping...",826262311560216578,0,Politics,"at that time, the only ""stepping over the line...",54
1194635634960478208,1194635634960478208,@MarkSZaidEsq It's not a coup. It's a Constit...,826262311560216578,0,Politics,it's not a coup. it's a constitutional procedu...,47
1194636052096413696,1194636052096413696,"@MarkSZaidEsq BTW, I've read your client HOSTE...",826262311560216578,0,Politics,"btw, i've read your client hosted (and was in)...",45


In [ ]:
#bert base uncased pretrained tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)


In [ ]:
# NOTE: batch_encode_plus adds [CLS], [SEP] to the text
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_mask=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        truncation=True,
        padding='longest',
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [ ]:
# BUILD THE MODEL
no_of_classes = 5
# seed for environmental stability 
# tf.random.set_seed(0) 

In [ ]:
def build_model(transformer, loss='categorical_crossentropy', max_len=512):
    input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    #adding dropout layer
    # x = tf.keras.layers.Dropout(0.3, seed=0)(cls_token)
    x = tf.keras.layers.Dropout(0.3)(cls_token)
    #using a dense layer of 11 neurons as the number of unique categories is 11. 
    #out = tf.keras.layers.Dense(11, activation='softmax')(x)
    out = tf.keras.layers.Dense(no_of_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=input_word_ids, outputs=out)
    #using categorical crossentropy as the loss as it is a multi-class classification problem
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss=loss, metrics=['accuracy'])
    return model

In [ ]:
#building the model on tpu
with strategy.scope():
    transformer_layer = transformers.TFAutoModel.from_pretrained('bert-base-uncased')
    model = build_model(transformer_layer, max_len=99)
model.summary()
# Save the initial Neural Net weights (to use for resetting the model in between cross validation runs)
init_weights = model.get_weights()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 99)]             0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  109482240
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             99, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                             

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
#X_train,X_test ,y_train,y_test = train_test_split(df['CleanTweet'], df['Target'], random_state = 1, stratify=df['Target'], test_size = 0.2)
X_train = df['CleanTweet']
y_train = df['Target']

In [ ]:
runTweak = "TF2.10, mini cleanup, remove 1st mention"

In [ ]:
runID = dt.today().strftime('%Y%m%d%H%M')
# 5-Fold Cross Validation
kf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True) 
# kf = StratifiedKFold(n_splits=5)
# kf = StratifiedKFold(n_splits=2, random_state=0, shuffle=True)
k=1
df_tr_hist = pd.DataFrame(columns=['RunID','RunTweak','Model','Loss','Accuracy'])
df_val_result = pd.DataFrame(columns=['ValID','ValType','Model','Accuracy','Precision','Recall','F1'])
for tr_idx, val_idx in kf.split(X_train, y_train):
  print(f"\nTraining for Cross Val: {k} ")
  print(f"Indexes for tr {tr_idx} and val {val_idx}")
  X_tr, X_val = X_train[tr_idx], X_train[val_idx]
  y_tr, y_val = y_train[tr_idx], y_train[val_idx]
  print("Encoding y_tr and y_val...")
  ytr_encoded = tf.keras.utils.to_categorical(y_tr, num_classes=no_of_classes,dtype = 'int32')
  yval_encoded = tf.keras.utils.to_categorical(y_val, num_classes=no_of_classes,dtype = 'int32')
  print("Encoding X_tr and X_val...")
  Xtr_encoded = regular_encode(X_tr.astype('str'), tokenizer, maxlen=99)
  Xval_encoded = regular_encode(X_val.astype('str'), tokenizer, maxlen=99)
  print(f"Shapes for X_tr {Xtr_encoded.shape}, y_tr {ytr_encoded.shape}, X_val {Xval_encoded.shape} and y_val {yval_encoded.shape}")
  
  print("creating the tr and val datasets...")
  epochs_no = 3
  BATCH_SIZE = 32*strategy.num_replicas_in_sync
  AUTO = tf.data.experimental.AUTOTUNE 
  train_dataset = tf.data.Dataset.from_tensor_slices((Xtr_encoded, ytr_encoded))
  test_dataset = tf.data.Dataset.from_tensor_slices(Xval_encoded)

  epochs_no = 10
  #creating the training and testing dataset.
  train_dataset = (
      tf.data.Dataset
      .from_tensor_slices((Xtr_encoded, ytr_encoded))
      .repeat()
      .shuffle(2048)
      .batch(BATCH_SIZE)
      .prefetch(AUTO)
  )
  test_dataset = (
      tf.data.Dataset
      .from_tensor_slices(Xval_encoded)
      .batch(BATCH_SIZE)
  )

  #training for the epochs
  n_steps = Xtr_encoded.shape[0] // BATCH_SIZE
  train_history = model.fit(
      train_dataset,
      steps_per_epoch=n_steps,
      epochs=epochs_no
  )

  #store the training history log
  df_tr_hist.loc[len(df_tr_hist)] = [runID, runTweak, 'BERT', train_history.history['loss'], train_history.history['accuracy']]

  #Making predictions for val
  preds = model.predict(test_dataset,verbose = 1)
  pred_classes = np.argmax(preds, axis = 1)
  
  #Mapping the encoded output to actual categories...
  # write result for decomposed SMR
  actual_category = np.argmax(yval_encoded, axis = 1) 
  acc = sklearn.metrics.accuracy_score(actual_category,pred_classes)
  print("Generating decomposed SMR performance metrics ..." )
  de_prec = precision_score(actual_category,pred_classes, average = 'weighted')
  de_recall = recall_score(actual_category,pred_classes, average = 'weighted')
  de_f1 = f1_score(actual_category,pred_classes, average = 'weighted')

  print(f"Decomposed SMR Acc: {acc*100}, Prec: {de_prec*100}, Recall: {de_recall*100}, F1 {de_f1*100}")
  df_val_result.loc[len(df_val_result)] = [k, 'Decomposed', 'BERT', acc*100, de_prec*100, de_recall*100, de_f1*100]

  # p_r_f1 = precision_recall_fscore_support(actual_category, pred_classes, average = 'weighted')
  # print(f"Decomposed SMR Acc: {acc*100}, Prec: {p_r_f1[0]*100}, Recall: {p_r_f1[1]*100}, F1 {p_r_f1[2]*100}")
  # df_val_result.loc[len(df_val_result)] = [k, 'Decomposed', '', acc*100, p_r_f1[0]*100, p_r_f1[1]*100, p_r_f1[2]*100]

  print("Generating recomposed SMR accuracy...")
  list_X_val_idx = X_val.index.tolist()
  result_df = pd.DataFrame({'idx':list_X_val_idx, 'actual_category':actual_category, 'predicted_category':pred_classes})
  df_sup_result = result_df.merge(df, left_on='idx', right_on='TID')
  df_sup_agg_result = df_sup_result.groupby(['InReplyTo','predicted_category']).size().reset_index(name='counts').sort_values(['InReplyTo','counts'], ascending=[True,False])
  df_rec = df[df.InReplyTo.isna()]

  # Recompose the SMR based on the count of aggregated supporting tweet predictions
  df_rec_result = pd.DataFrame(columns=['TID','Actual','Predicted'])
  for i in range(0,len(df_rec)):
    rec_TID = df_rec.iloc[i]['TID']
    rec_cat = int(df_rec.iloc[i]['Target'])
    df_temp = df_sup_agg_result[df_sup_agg_result.InReplyTo == rec_TID].sort_values('counts',ascending=False)
    if len(df_temp) > 0:
      rec_pred = int(df_temp[:1]['predicted_category'].values)
      df_rec_result.loc[len(df_rec_result)] = [rec_TID,rec_cat,rec_pred]

  # write result for recomposed SMR
  rec_actual = df_rec_result['Actual'].to_numpy().astype(int)
  rec_pred = df_rec_result['Predicted'].to_numpy().astype(int)
  rec_acc = sklearn.metrics.accuracy_score(rec_actual, rec_pred)
  rec_prec = precision_score(rec_actual, rec_pred, average = 'weighted')
  rec_recall = recall_score(rec_actual, rec_pred, average = 'weighted')
  rec_f1 = f1_score(rec_actual, rec_pred, average = 'weighted')

  print(f"Recomposed SMR Acc: {rec_acc*100}, Prec: {rec_prec*100}, Recall: {rec_recall*100}, F1 {rec_f1*100}")
  df_val_result.loc[len(df_val_result)] = [k, 'Recomposed', 'BERT', rec_acc*100, rec_prec*100, rec_recall*100, rec_f1*100]

  # rec_p_r_f1 = precision_recall_fscore_support(rec_actual, rec_pred, average = 'weighted')
  # print(f"Recomposed SMR Acc: {rec_acc*100}, Prec: {rec_p_r_f1[0]*100}, Recall: {rec_p_r_f1[1]*100}, F1 {rec_p_r_f1[2]*100}")
  # df_val_result.loc[len(df_val_result)] = [k, 'Recomposed', '', rec_acc*100, rec_p_r_f1[0]*100, rec_p_r_f1[1]*100, rec_p_r_f1[2]*100]

  # Generate confusion matrix
  rec_cm = confusion_matrix(rec_actual, rec_pred)
  df_rec_cm = pd.DataFrame(rec_cm)
  df_rec_cm.to_csv("/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/results/XLNet_GTr_CM_" + str(k) + " " + runID +".csv")

  # Reset the model by restoring the initital model weights to prevent overtraining
  model.set_weights(init_weights)
  k=k+1

# Save the training history result
df_tr_hist.to_csv("/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/results/BERT_GTr_TrainHist_" + runID +".csv")

# Save the cross validation result
df_val_result = df_val_result.round(decimals=2)
df_val_result.to_csv("/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/results/BERT_GTr_CV_" + runID +".csv")

# Save RunID,Model,Accuracy,Precision,Recall,F1 for the Recomposed SMR
df_re_val_result = df_val_result[df_val_result.ValType=='Recomposed']
no_of_runs = len(df_re_val_result)
df_GTxM_Clf_CV_results = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/results/GTxM_Clf_CV_results.csv")
avg_Accuracy = np.sum(df_re_val_result['Accuracy']) / no_of_runs
avg_Precision = np.sum(df_re_val_result['Precision']) / no_of_runs
avg_Recall = np.sum(df_re_val_result['Recall']) / no_of_runs
avg_F1 = np.sum(df_re_val_result['F1']) / no_of_runs
df_GTxM_Clf_CV_results.loc[len(df_GTxM_Clf_CV_results)] = [runID, 'BERT', avg_Accuracy, avg_Precision, avg_Recall, avg_F1]
df_GTxM_Clf_CV_results.to_csv("/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/results/GTxM_Clf_CV_results.csv", index=False)


Training for Cross Val: 1 
Indexes for tr [     0      1      2 ... 101450 101451 101452] and val [     4      8     10 ... 101427 101429 101453]
Encoding y_tr and y_val...
Encoding X_tr and X_val...
Shapes for X_tr (81163, 99), y_tr (81163, 5), X_val (20291, 99) and y_val (20291, 5)
creating the tr and val datasets...
Epoch 1/10


317/317 [==============================] - 136s 187ms/step - loss: 0.7955 - accuracy: 0.7190
Epoch 2/10
317/317 [==============================] - 59s 187ms/step - loss: 0.5303 - accuracy: 0.8234
Epoch 3/10
317/317 [==============================] - 59s 186ms/step - loss: 0.4061 - accuracy: 0.8663
Epoch 4/10
317/317 [==============================] - 59s 186ms/step - loss: 0.3348 - accuracy: 0.8900
Epoch 5/10
317/317 [==============================] - 59s 186ms/step - loss: 0.2767 - accuracy: 0.9075
Epoch 6/10
317/317 [==============================] - 59s 187ms/step - loss: 0.2243 - accuracy: 0.9259
Epoch 7/10
317/317 [==============================] - 59s 186ms/step - loss: 0.1784 - accuracy: 0.9417
Epoch 8/10
317/317 [==============================] - 59s 186ms/step - loss: 0.1399 - accuracy: 0.9541
Epoch 9/10
317/317 [==============================] - 59s 186ms/step - loss: 0.1090 - accuracy: 0.9651
Epoch 10/10
80/80 [==============================] - 19s 172ms/step
Generating deco